<a href="https://colab.research.google.com/github/einsteinxx/UCLA_BIOENG_596_FALL_21/blob/main/rebuild_raw_ultrasound_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
import math #for ceil function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files
import random #used to select random slice for patches
import cv2
import re #regexp for BUSI files
from skimage import exposure  #image equalization
import torchvision.transforms as transforms #for imagenet specific stage2 normalization

################################################################################
#ULTRASOUND NEEDS
from bs4 import BeautifulSoup #to read the XML annotation files

import PIL
#from PIL import Image
# Open the image form working directory
#image = Image.open(full_file)

from matplotlib import image
from ast import literal_eval #used to break out bounding boxes from strings
import time


from numpy import clip

################################################################################

eps = 1e-16 #keep div by 0 away


from google.colab.patches import cv2_imshow
#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5
#for beautifulsoup
!pip3 -q install beautifulsoup4

#for lmxl parser
!pip3 -q install lxml

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image array
import shutil #copy file libraries

################################################################################
# Load data from google drive
################################################################################
drive.mount('/content/gdrive')

#
# Setup of input and output drives
#

#
# Choose training sets to use
#
training_set = 0 #0=UCLA US, 1 = UCLA + BUSI, 2 = BUSI

#storage areas 
data_dir = '/content/gdrive/Shareddrives/BreastUS'
local_dir = '/content/gdrive/My Drive/BreastUS' #for local storage
model_dir = '/content/gdrive/My Drive/BreastUS/MODEL_SAVE'
tensorboard_dir =  '/content/gdrive/My Drive/BreastUS/TENSORBOARD_SUMMARIES'


#
# UCLA specific data folders
#
csv_dir = os.path.join(data_dir,'Annotated data')

annotated_dir = os.path.join(data_dir,'Annotated data')

data_files = os.listdir(annotated_dir)
label_data_dir = os.path.join(annotated_dir,'LabelMe_3.0_format_updated')
label_files = os.listdir(label_data_dir)

#
# Alternative set of Ultrasound data
#
busi_main_dir = '/content/gdrive/My Drive/BreastUS/BUSI_DATA' #downloaded BUSI

#output converted BUSI images go in here:
norm_dir = '0TO1NORM'  #-2 to 2 norm method is N2TO2_NORM\

busi_converted_main_dir = os.path.join('/content/gdrive/My Drive/BreastUS/',norm_dir,'BUSI_DATA_CONTRAST_EQUALIZED')
ucla_converted_main_dir = os.path.join('/content/gdrive/My Drive/BreastUS/',norm_dir,'UCLA_DATA_CONTRAST_EQUALIZED')



     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 256 kB 5.3 MB/s 
Mounted at /content/gdrive


# Extract Annotations from XML

In [2]:


def extract_bb(full_file):
    corner = []
 

    #full_file = xml_list[30]
    # Reading the data inside the xml file to a variable under the name  data
    with open(full_file, 'r') as f:
        data = f.read() 


    # Passing the stored data inside the beautifulsoup parser 
    bs_data = BeautifulSoup(data, 'xml') 

    # Finding all instances of tag   
    nrows = bs_data.find_all('nrows') 
    ncols = bs_data.find_all('ncols') 
    boundingbox = bs_data.find_all('bounding_box')
    xx = bs_data.find_all('x')
    yy = bs_data.find_all('y')
    #print(nrows) 

    #for ii in [nrows,ncols]:
    #    print(ii[0])
    #    print(ii[0].get_text())


    xpoints=[]
    ypoints=[]

    for item in xx:
        xpoints.append(int(float(item.text)))
    for item in yy:
        ypoints.append(int(float(item.text)))

    max_points = 4
    if ((len(xpoints)<max_points) or (len(ypoints)<max_points)):
        print('Not enough annotations ',len(xpoints), len(ypoints))

    else: #proceed to make new corner positions

        
        for num in range(0,4):
            corner.append((ypoints[num],xpoints[num]))

        #for item in boundingbox:
        #    pt = item.find('pt').text
        #    print('pt is ',pt)
    return corner
'''
    stop
########### Title #############
required0 = soup.find_all("title")
title = []
for i in required0:
    title.append(i.get_text())

# Using find() to extract attributes of the first instance of the tag 
b_name = bs_data.find('child', {'name':'Acer'}) 
print(b_name) 

# Extracting the data stored in a specific attribute of the `child` tag 
value = b_name.get('qty') 
print(value)
'''

'\n    stop\n########### Title #############\nrequired0 = soup.find_all("title")\ntitle = []\nfor i in required0:\n    title.append(i.get_text())\n\n# Using find() to extract attributes of the first instance of the tag \nb_name = bs_data.find(\'child\', {\'name\':\'Acer\'}) \nprint(b_name) \n\n# Extracting the data stored in a specific attribute of the `child` tag \nvalue = b_name.get(\'qty\') \nprint(value)\n'

In [3]:
def get_csv_data(filename):
    fields = []
    rows = []

    # reading csv file
    with open(filename, 'r') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        
        # extracting field names through first row
        fields = next(csvreader)

        # extracting each data row one by one
        for row in csvreader:
            rows.append(row)


        # lines present
        print("found rows: %d"%(csvreader.line_num))
    return fields, rows

In [ ]:
if (0):
    from shutil import copytree

    batch1_dir = os.path.join(annotated_dir, 'unzipped_updated')
    batch2_dir = os.path.join(annotated_dir,'unzipped_Batch2')
    output_dir = os.path.join(annotated_dir,'unzipped_updated_plus_Batch2')

    batch1_list = os.listdir(batch1_dir)
    batch2_list =os.listdir(batch2_dir)
    full_input_list = []
    for folder_name in batch1_list:
        input_dir_name = os.path.join(batch1_dir,folder_name)
        full_input_list.append(input_dir_name)

    for folder_name in batch2_list:
        input_dir_name = os.path.join(batch2_dir,folder_name)  
        full_input_list.append(input_dir_name)

    print(len(full_input_list))

    docopy = 0
    if (docopy == 1):
        for input_folder in full_input_list:
            #print(input_folder)
            output_subdir = os.path.join(output_dir,os.path.basename(input_folder))
            shutil.copytree(input_folder,output_subdir)
            print('copied input_folder to output: ', input_folder)


    print('Finished copying folders')
    #shutil.copytree("sourcedir", "destination")

In [ ]:
data_files
#annotated_dir for batch2 area
b2_dir = os.path.join(annotated_dir,'unzipped_Batch2')

csv1 = 'annotations_updated_final.csv'
csv_file = os.path.join(csv_dir,csv1)
csv2='case_list_batch2_w_diagnosis.csv' #batch2 csv info
csv2_file = os.path.join(data_dir,csv2)

#
# Get Batch2 CSV information to assign to new rows
#
fields2, rows2 = get_csv_data(csv2_file)
fields, rows = get_csv_data(csv_file)

#patient directories
pdir = os.listdir(b2_dir)

patient_data={}
for pt_list in pdir:
    print(pt_list)
    patient_data[pt_list] ={}
    

    patient = os.path.join(b2_dir, pt_list)

    #--- add in the remaining folders
    #These batch2 datasets have a default/default path added
    default = os.path.join(patient,'default','default')

    if (os.path.exists(default) == True):
        print('!!!! double default found !!!!')
        pass
    else:
        #some folders may not have default/default, so remove one of them and go
        default = os.path.join(patient,'default')

    files = os.listdir(default)
    print(default)
    xml_list =[]
    for ii in files:
        
        if ('.xml' in ii):
            temp = ii.split('_')
            number = re.findall('\d+',temp[1])
            frame_id = int(number[0])
            
            
            full_xml_file = os.path.join(default,ii)
            corner_data = extract_bb(full_xml_file)
            patient_data[pt_list][frame_id] = corner_data
            image_ii = ii.replace('.xml', '.png')
            patient_data[pt_list]['image_path'] = os.path.join(default,image_ii)
            print(corner_data)
            #xml_list.append(os.path.join(default,ii))
            



In [37]:

for pt in patient_data.keys():
    frame_id = 0
    print(pt, patient_data[pt])
    print(pt)

    #
    # Get coded mrm, coded accession, and video id
    #
    name = pt.split('_')
    print(name)
    mrm = name[0] +'_'+ name[1]
    mrm = mrm.upper()
    acc = name[2] + '_' + name[3]
    acc = acc.upper()
    video_id = pt
    image_path = patient_data[pt]['image_path']
    box = patient_data[pt][frame_id]

    #
    # Get Info from Batch2 CSV
    #
    for row_vals in rows2:
        if (mrm in row_vals[0]):
            if (acc in row_vals[1]):
                print('found a matching mrm/acc')
                first50 = row_vals[4]
                finaldiag = row_vals[2]
                bxsite = row_vals[3]

    if (len(first50) == 0):
        #something went wrong with the comparison, kill this process
        print('failed csv compare')
        stop

    print(mrm)
    print(acc)
    print(video_id)
    row_data = [0,mrm, acc, video_id, image_path, box, finaldiag, bxsite, 'N/A',first50]
    print(row_data)
    print(patient_data[pt].keys())
    stop

    #
    # Get the frame ids (keys) in order
    #
    klist=[]
    for ii in patient_data[pt].keys():
        klist.append(ii)
        #print(ii)
        #print(patient_data[pt][ii])
    klist.sort()
    print(klist)

    for ii in klist:
        print(ii,patient_data[pt][ii])

    stop


    # open the file in the write mode
    f = open('path/to/csv_file', 'w')

    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(row)

    # close the file
    f.close()

1_87j68lw4_a_j9fg822f_2.mp4-2022_01_31_00_50_55-labelme 3.0.zip {10: [(188, 336), (188, 547), (357, 547), (357, 336)], 'image_path': '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_87j68lw4_a_j9fg822f_2.mp4-2022_01_31_00_50_55-labelme 3.0.zip/default/frame_000067.png', 12: [(188, 336), (188, 547), (357, 547), (357, 336)], 4: [(214, 353), (214, 496), (359, 496), (359, 353)], 50: [(199, 348), (199, 529), (334, 529), (334, 348)], 45: [(199, 360), (199, 551), (339, 551), (339, 360)], 15: [(188, 319), (188, 547), (357, 547), (357, 319)], 62: [], 46: [(199, 350), (199, 551), (332, 551), (332, 350)], 25: [(176, 331), (176, 557), (341, 557), (341, 331)], 8: [(196, 357), (196, 533), (359, 533), (359, 357)], 18: [(181, 327), (181, 547), (357, 547), (357, 327)], 64: [], 56: [], 1: [(247, 364), (247, 496), (359, 496), (359, 364)], 24: [(176, 331), (176, 557), (343, 557), (343, 331)], 61: [], 35: [(176, 331), (176, 557), (343, 557), (343, 331)], 3: [(225, 354), (225, 496), (

NameError: ignored

In [34]:
len([])

0

In [15]:
print(fields)
print(fields2)
print(len(rows))
print(rows[0])
print('---------------------')
for rr in rows2:
    print(rr)

['', 'coded_mrn', 'coded_accession', 'video_id', 'frame_id', 'image_path', 'bounding box', 'Final Diagnosis', 'Bx Site', 'Final Diagnosis.1', 'First 50']
['\ufeffcoded_mrn', 'coded_accession', 'Final Diagnosis', 'Bx Site', 'Coded Diagnosis']
8842
['0', '1_076UP782', 'A_DCC4675O', '1_076up782_a_dcc4675o_0.mp4-2021_09_03_16_33_59-labelme 3.0.zip', '0', 'drive/MyDrive/Annotated data/unzipped_updated/1_076up782_a_dcc4675o_0.mp4-2021_09_03_16_33_59-labelme 3.0.zip/default/frame_000000.PNG', '[]', 'Breast tissue with pseudoangiomatous stromal hyperplasia (PASH)', 'Left Breast, 11:00, 1 cm FTN', 'Breast tissue with pseudoangiomatous stromal hyperplasia (PASH)', 'B']
---------------------
['1_Z97OU004', 'A_89K78944\n', 'Complex fibroadenoma with sclerosing adenosis, apocrine metaplasia and microcysts', 'Right Breast, mass, 12:00, 5cm FTN', 'B']
['1_7S6UGZFF', 'A_32N0Q57D\n', 'Fibroepithelial lesion, most consistent with fibroadenoma ', 'Right Breast, mass, 11:00, 5cm FTN', 'B']
['1_4SA4ZUV9', 

In [6]:
temp= ii.split('_')
number = re.findall('\d+',temp[1])
number = int(number[0])
print(ii)
print(number)

frame_000010.xml
10


In [3]:
#!ls -lart '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_pq8nn45n_a_x4g5x404_2.mp4-2022_01_10_00_50_42-labelme 3.0.zip'

!ls -lart '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_en9w0q3l_a_z4tp3894_2.mp4-2022_01_10_00_50_31-labelme 3.0.zip'

total 4
drwx------ 2 root root 4096 Mar 24 10:09 default


'\n    stop\n########### Title #############\nrequired0 = soup.find_all("title")\ntitle = []\nfor i in required0:\n    title.append(i.get_text())\n\n# Using find() to extract attributes of the first instance of the tag \nb_name = bs_data.find(\'child\', {\'name\':\'Acer\'}) \nprint(b_name) \n\n# Extracting the data stored in a specific attribute of the `child` tag \nvalue = b_name.get(\'qty\') \nprint(value)\n'

In [ ]:
xml_list

['/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default/frame_000010.xml',
 '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default/frame_000012.xml',
 '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default/frame_000086.xml',
 '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default/frame_000004.xml',
 '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default/frame_000088.xml',
 '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_9glo6t2y_a_p7y22iu8_2.mp4-2022_02_04_20_41_47-labelme 3.0.zip/default/default

In [ ]:
    print('starting UCLA conversion')
    dbprint =0
    full_pickle_list=[] #store files sent to pickle

    print('--- Loading UCLA Data Files and Categorizing')
    data_dirs = os.listdir(busi_main_dir)
    print('UCLA directories: ', data_dirs)



    annotation_fields, annotation_rows= get_ucla_csv(csv_dir)
    array_rows, mrn, accession, video_id, frame_id, image_path, bounding_box, \
        diagnosis, biopsy_site, diagnosis2, first50 = sort_UCLA_csv(annotation_rows, annotation_fields)


    ############################################################################
    # Get Folder data
    ############################################################################
    full_file_list=[]

    for file_path in image_path:
        #print(file_path)

        filename = os.path.basename(file_path) 
        [_,fpath] =file_path.split('drive/MyDrive/Annotated data/')
        full_file = os.path.join(annotated_dir,fpath)
        full_file_list.append(full_file)
    print('Number of US files: ', len(full_file_list))
    original_file_length = len(full_file_list)

    video_stats(training_set, video_id, bounding_box)

    ############################################################################
    #
    # Default string subsets for replacement for UCLA folders
    # -to replace the CSV given names
    old_string = '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_updated'
    new_string = ucla_converted_main_dir

